In [1]:
import pandas as pd
import numpy as np
import plotly.express as px
import plotly.graph_objects as go
from jupyter_dash import JupyterDash
from dash import dcc, html, Input, Output

file_path = "../../data/processed/medical_equipment_utilization_synthetic_cleaned(in).csv"

raw_df = pd.read_csv(file_path)
raw_df.columns = [c.strip().replace(" ", "_") for c in raw_df.columns]

df = raw_df.dropna(axis=1, how="all").copy()

currency_cols = [
    "Cost_per_procedure",
    "Daily_Operating_Cost",
    "Procedure_Revenue",
    "Net-Profit_(daily)",
]
for col in currency_cols:
    if col in df.columns:
        df[col] = (
            df[col]
            .astype(str)
            .str.replace("KES", "", case=False, regex=False)
            .str.replace(",", "", regex=False)
            .str.replace(" ", "", regex=False)
        )
        df[col] = pd.to_numeric(df[col], errors="coerce")

numeric_cols = [
    "Available_Hours",
    "Actual_Operating_Hours",
    "Utilization_Rate",
    "Number_of_Procedures",
]
for col in numeric_cols:
    if col in df.columns:
        df[col] = pd.to_numeric(df[col], errors="coerce")

if "Date" in df.columns:
    df["Date"] = pd.to_datetime(df["Date"], errors="coerce")

by_type = (
    df.groupby("Equipment_Type")
    .agg(
        avg_utilization_rate=("Utilization_Rate", "mean"),
        avg_procedures_per_day=("Number_of_Procedures", "mean"),
        total_procedures=("Number_of_Procedures", "sum"),
        avg_operating_hours=("Actual_Operating_Hours", "mean"),
        total_revenue=("Procedure_Revenue", "sum"),
        total_operating_cost=("Daily_Operating_Cost", "sum"),
        total_net_profit=("Net-Profit_(daily)", "sum"),
        n_days=("Equipment_ID", "count"),
    )
)
by_type["roi_%"] = 100 * by_type["total_net_profit"] / by_type["total_operating_cost"]
by_type["profit_margin_%"] = 100 * by_type["total_net_profit"] / by_type["total_revenue"]
by_type["avg_profit_per_day"] = by_type["total_net_profit"] / by_type["n_days"]

by_dept = (
    df.groupby("Department")
    .agg(
        avg_utilization_rate=("Utilization_Rate", "mean"),
        total_procedures=("Number_of_Procedures", "sum"),
        total_revenue=("Procedure_Revenue", "sum"),
        total_operating_cost=("Daily_Operating_Cost", "sum"),
        total_net_profit=("Net-Profit_(daily)", "sum"),
        n_days=("Equipment_ID", "count"),
    )
)
by_dept["roi_%"] = 100 * by_dept["total_net_profit"] / by_dept["total_operating_cost"]
by_dept["profit_margin_%"] = 100 * by_dept["total_net_profit"] / by_dept["total_revenue"]
by_dept["avg_profit_per_day"] = by_dept["total_net_profit"] / by_dept["n_days"]

if "Month" in df.columns:
    by_month = (
        df.groupby("Month")
        .agg(
            avg_utilization_rate=("Utilization_Rate", "mean"),
            total_procedures=("Number_of_Procedures", "sum"),
            total_net_profit=("Net-Profit_(daily)", "sum"),
        )
        .sort_values("avg_utilization_rate", ascending=False)
    )
else:
    by_month = None

print("Data prepared for dashboard")


ModuleNotFoundError: No module named 'plotly'

In [ ]:
def make_eda_content():
    fig_util = px.histogram(
        df,
        x="Utilization_Rate",
        nbins=30,
        title="Utilization Rate Distribution",
        marginal="box",
    )

    fig_hours = px.scatter(
        df,
        x="Available_Hours",
        y="Actual_Operating_Hours",
        color="Equipment_Type",
        hover_data=["Equipment_ID", "Department"],
        title="Available vs Actual Hours",
    )

    eq_counts = df["Equipment_Type"].value_counts().reset_index()
    eq_counts.columns = ["Equipment_Type", "Count"]
    fig_eq = px.bar(eq_counts, x="Equipment_Type", y="Count", title="Records by Equipment Type")
    fig_eq.update_layout(xaxis_tickangle=-45)

    dept_fig = None
    if "Department" in df.columns:
        dept_counts = df["Department"].value_counts().reset_index()
        dept_counts.columns = ["Department", "Count"]
        dept_fig = px.bar(dept_counts, x="Department", y="Count", title="Records by Department")
        dept_fig.update_layout(xaxis_tickangle=-30)

    graphs = [dcc.Graph(figure=fig_util), dcc.Graph(figure=fig_hours), dcc.Graph(figure=fig_eq)]
    if dept_fig is not None:
        graphs.append(dcc.Graph(figure=dept_fig))

    return graphs


def make_roi_content():
    fig_roi_type = px.bar(
        by_type.reset_index(),
        x="Equipment_Type",
        y="roi_%",
        hover_data=["profit_margin_%", "avg_profit_per_day"],
        title="ROI (%) by Equipment Type",
    )
    fig_roi_type.update_layout(xaxis_tickangle=-45)

    fig_profit_type = px.bar(
        by_type.reset_index(),
        x="Equipment_Type",
        y="avg_profit_per_day",
        title="Average Profit per Day (KES) by Equipment Type",
    )
    fig_profit_type.update_layout(xaxis_tickangle=-45)

    fig_roi_dept = px.bar(
        by_dept.reset_index(),
        x="Department",
        y="roi_%",
        hover_data=["profit_margin_%", "avg_profit_per_day"],
        title="ROI (%) by Department",
    )
    fig_roi_dept.update_layout(xaxis_tickangle=-30)

    fig_profit_dept = px.bar(
        by_dept.reset_index(),
        x="Department",
        y="avg_profit_per_day",
        title="Average Profit per Day (KES) by Department",
    )
    fig_profit_dept.update_layout(xaxis_tickangle=-30)

    return [
        dcc.Graph(figure=fig_roi_type),
        dcc.Graph(figure=fig_profit_type),
        dcc.Graph(figure=fig_roi_dept),
        dcc.Graph(figure=fig_profit_dept),
    ]


def make_utilization_content():
    fig_util_type = px.bar(
        by_type.reset_index(),
        x="Equipment_Type",
        y="avg_utilization_rate",
        hover_data=["avg_operating_hours", "avg_procedures_per_day"],
        title="Average Utilization Rate by Equipment Type",
    )
    fig_util_type.update_layout(xaxis_tickangle=-45)

    fig_proc_type = px.bar(
        by_type.reset_index(),
        x="Equipment_Type",
        y="avg_procedures_per_day",
        title="Average Procedures per Day by Equipment Type",
    )
    fig_proc_type.update_layout(xaxis_tickangle=-45)

    fig_util_dept = px.bar(
        by_dept.reset_index(),
        x="Department",
        y="avg_utilization_rate",
        title="Average Utilization Rate by Department",
    )
    fig_util_dept.update_layout(xaxis_tickangle=-30)

    content = [
        dcc.Graph(figure=fig_util_type),
        dcc.Graph(figure=fig_proc_type),
        dcc.Graph(figure=fig_util_dept),
    ]

    if by_month is not None:
        fig_util_month = px.bar(
            by_month.reset_index(),
            x="Month",
            y="avg_utilization_rate",
            title="Average Utilization Rate by Month",
        )
        fig_util_month.update_layout(xaxis_tickangle=-30)
        content.append(dcc.Graph(figure=fig_util_month))

    return content


In [ ]:
app = JupyterDash(__name__)

tab_style = {"padding": "10px", "fontWeight": "600"}
tab_selected_style = {"padding": "10px", "fontWeight": "600", "backgroundColor": "#e0f7fa"}

app.layout = html.Div(
    [
        html.H2("Medical Equipment Utilization - Interactive Dashboard"),
        dcc.Tabs(
            id="dashboard-tabs",
            value="tab-eda",
            children=[
                dcc.Tab(label="EDA", value="tab-eda", style=tab_style, selected_style=tab_selected_style),
                dcc.Tab(label="ROI", value="tab-roi", style=tab_style, selected_style=tab_selected_style),
                dcc.Tab(label="Utilization", value="tab-util", style=tab_style, selected_style=tab_selected_style),
            ],
        ),
        html.Div(id="tab-content"),
    ]
)


@app.callback(Output("tab-content", "children"), Input("dashboard-tabs", "value"))
def render_tab(tab):
    if tab == "tab-roi":
        return make_roi_content()
    if tab == "tab-util":
        return make_utilization_content()
    return make_eda_content()


# Run the app inline inside Jupyter (works with VS Code notebooks too)
app.run_server(mode="inline", height=900)
